In [1]:
## Getting to Main directory
import os
os.chdir("../")

In [2]:
# loading secret key
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pandas as pd

In [4]:
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI

from typing import List
from langchain.docstore.document import Document


e:\projects\AI research assistant\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading APIs

In [5]:
load_dotenv()
gemini_api_key=os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [6]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter


In [7]:
# loading app details csv
csv_file="Data/app_details.csv"

## Loading the data

In [8]:
loader = CSVLoader(file_path="Data/app_details.csv")
data = loader.load()

### Loading CSV as Dataframe to see the data

In [9]:
df = pd.read_csv(csv_file)

In [32]:
df.head()

,appId_or_package_name,title,description,summary,installs,score,ratings,developer,categories,icon,headerImage,contentRating,contentRatingDescription,released,lastUpdatedOn,version,url
0,com.supercell.clashofclans,Clash of Clans,Join millions of players worldwide as you buil...,"Epic combat strategy game. Build your village,...","500,000,000+",4.500079,61153372.0,Supercell,"[{'name': 'Strategy', 'id': 'GAME_STRATEGY'}, ...",https://play-lh.googleusercontent.com/LByrur1m...,https://play-lh.googleusercontent.com/D6u3M1r7...,Rated for 7+,Mild Violence,"Sep 30, 2013","May 3, 2024",16.253.25,https://play.google.com/store/apps/details?id=...
1,com.supercell.clashroyale,Clash Royale,Enter the Arena! Build your Battle Deck and ou...,"Clash Royale is a real-time, head-to-head batt...","500,000,000+",4.178203,37533792.0,Supercell,"[{'name': 'Strategy', 'id': 'GAME_STRATEGY'}, ...",https://play-lh.googleusercontent.com/z950eFx-...,https://play-lh.googleusercontent.com/s9qs3woa...,Rated for 7+,Mild Violence,"Mar 1, 2016","Apr 17, 2024",60256021,https://play.google.com/store/apps/details?id=...
2,com.dts.freefiremax,Free Fire MAX,[Mechadrake Incoming]\r\nThe fierce Mechadrake...,MAX Battle Royale Experience,"100,000,000+",4.301307,22476456.0,Garena International I,"[{'name': 'Action', 'id': 'GAME_ACTION'}, {'na...",https://play-lh.googleusercontent.com/T8RuSQDF...,https://play-lh.googleusercontent.com/PTF7b77h...,Rated for 12+,Moderate Violence,"Sep 18, 2021","Apr 7, 2024",2.104.1,https://play.google.com/store/apps/details?id=...
3,com.topgamesinc.evony,Evony: The King's Return,Shoot and engage in battles！\r\nBuild your cit...,"Move and Shoot! Build Your Empire, Explore Dif...","100,000,000+",3.547859,707011.0,TG Inc.,"[{'name': 'Strategy', 'id': 'GAME_STRATEGY'}, ...",https://play-lh.googleusercontent.com/_AnSpAUp...,https://play-lh.googleusercontent.com/0WahHxuN...,Rated for 7+,Mild Violence,"Aug 10, 2016","May 17, 2024",NaN,https://play.google.com/store/apps/details?id=...
4,com.vizorapps.klondike,Klondike Adventures,Do you like travelling to odd places? 🗺 Fancy ...,Farming games: Harvest crops and Grow your own...,"50,000,000+",4.050404,1233575.0,VIZOR APPS LTD.,"[{'name': 'Simulation', 'id': 'GAME_SIMULATION...",https://play-lh.googleusercontent.com/jW61KFsl...,https://play-lh.googleusercontent.com/vyiC4Rx0...,Rated for 3+,NaN,"Apr 3, 2018","May 10, 2024",2.121.1,https://play.google.com/store/apps/details?id=...


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   appId_or_package_name     183 non-null    object 
 1   title                     183 non-null    object 
 2   description               183 non-null    object 
 3   summary                   183 non-null    object 
 4   installs                  162 non-null    object 
 5   score                     151 non-null    float64
 6   ratings                   151 non-null    float64
 7   developer                 183 non-null    object 
 8   categories                183 non-null    object 
 9   icon                      183 non-null    object 
 10  headerImage               183 non-null    object 
 11  contentRating             183 non-null    object 
 12  contentRatingDescription  97 non-null     object 
 13  released                  153 non-null    object 
 14  lastUpdate

## Chunking the data

In [11]:
# Transform data
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts = text_splitter.split_documents(data)

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [13]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest", api_key=gemini_api_key, temperature=0.3,  convert_system_message_to_human=True
)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

genai.configure(api_key=gemini_api_key)


## Upserting the Data

In [14]:
# Fill Vector DB
docsearch = FAISS.from_documents(texts, embeddings)
retriever=docsearch.as_retriever()

In [15]:
vector_store = FAISS.from_documents(texts, embedding=embeddings)
vector_store.save_local("faiss_index")
print("Successfully stored indexes")

Successfully stored indexes


In [16]:
new_db = FAISS.load_local(
        "faiss_index", embeddings, allow_dangerous_deserialization=True
    )

retriever1=new_db.as_retriever()

In [17]:
ans = docsearch.similarity_search("Which games may you recommend for adventure lover?")

ans[0]


Document(page_content='appId_or_package_name: com.goongames.DungeonsofDreadrock\ntitle: Dungeons of Dreadrock\ndescription: Puzzle your way through 100 handcrafted levels into the ancient depths of Dreadrock Mountain.\r\n\r\n"It\'s super creative" (IGN)\r\n"A masterpiece" (WATSON)\r\n"Beautifully designed" (PURENINTENDO)\r\n"Something really special" (MACWORLD)\nsummary: An immersive puzzle dungeon adventure - Winner of IGF 2022\ninstalls: 1,000,000+\nscore: 4.8072915\nratings: 35916.0\ndeveloper: Christoph Minnameier\ncategories: [{\'name\': \'Action\', \'id\': \'GAME_ACTION\'}, {\'name\': \'Action-adventure\', \'id\': None}, {\'name\': \'Casual\', \'id\': \'GAME_CASUAL\'}, {\'name\': \'Single player\', \'id\': None}, {\'name\': \'Stylized\', \'id\': None}, {\'name\': \'Offline\', \'id\': None}]\nicon: https://play-lh.googleusercontent.com/fu9gbEVB4YWx9AwEnkDsv1QlBf9iOIk7WlMXTuv1oJ2XVw4f_nKHS0Oc2DygGkKzVg\nheaderImage: https://play-lh.googleusercontent.com/0JGFClzQlrsu159aB2R0xaiITlZb

In [18]:
docs = retriever.get_relevant_documents("Which games may you recommend for adventure lover?")

docs[0]

Document(page_content='appId_or_package_name: com.goongames.DungeonsofDreadrock\ntitle: Dungeons of Dreadrock\ndescription: Puzzle your way through 100 handcrafted levels into the ancient depths of Dreadrock Mountain.\r\n\r\n"It\'s super creative" (IGN)\r\n"A masterpiece" (WATSON)\r\n"Beautifully designed" (PURENINTENDO)\r\n"Something really special" (MACWORLD)\nsummary: An immersive puzzle dungeon adventure - Winner of IGF 2022\ninstalls: 1,000,000+\nscore: 4.8072915\nratings: 35916.0\ndeveloper: Christoph Minnameier\ncategories: [{\'name\': \'Action\', \'id\': \'GAME_ACTION\'}, {\'name\': \'Action-adventure\', \'id\': None}, {\'name\': \'Casual\', \'id\': \'GAME_CASUAL\'}, {\'name\': \'Single player\', \'id\': None}, {\'name\': \'Stylized\', \'id\': None}, {\'name\': \'Offline\', \'id\': None}]\nicon: https://play-lh.googleusercontent.com/fu9gbEVB4YWx9AwEnkDsv1QlBf9iOIk7WlMXTuv1oJ2XVw4f_nKHS0Oc2DygGkKzVg\nheaderImage: https://play-lh.googleusercontent.com/0JGFClzQlrsu159aB2R0xaiITlZb

## Using RetrievalQA to make final query

In [19]:
from langchain.chains import RetrievalQA

In [20]:
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=retriever1)
qa.run({"query": "Which games may you recommend for adventure lover?"} )


e:\projects\AI research assistant\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Based on the provided descriptions, I would recommend these games for an adventure lover:\n\n* **Shovel Pirate:**  This game features a pirate on a quest to gather treasures, with platforming, puzzles, and traps to overcome. \n* **Machinarium:** This game follows a robot named Josef trying to save his girlfriend from the Black Cap Brotherhood gang. It features a unique visual style and puzzle-solving elements. \n* **Dungeons of Dreadrock:** While this game is primarily puzzle-focused, its 100 handcrafted levels through Dreadrock Mountain suggest a sense of exploration and adventure. \n'

In [21]:
qa.run({"query": "Which games may you recommend for adventure lover give me 10 recommendation?, give me image url too"} )


'Here are some adventure games for you:\n\n1. **Dungeons of Dreadrock**\n   - **Image URL:** https://play-lh.googleusercontent.com/fu9gbEVB4YWx9AwEnkDsv1QlBf9iOIk7WlMXTuv1oJ2XVw4f_nKHS0Oc2DygGkKzVg\n   - **Description:** Puzzle your way through 100 handcrafted levels in this immersive puzzle dungeon adventure.\n\n2. **Shovel Pirate**\n   - **Image URL:** https://play-lh.googleusercontent.com/CR8maPXYv56LEWjtLX9Nt9UcYqIINN_YIAvHWdSvDcvCuDYKj4Q2cgA07lbOKS6mDR8\n   - **Description:**  A cute platformer where you play as a pirate on a quest to gather treasures. Features fun puzzles, adorable characters, and clever traps.\n\n3. **Machinarium**\n   - **Image URL:** https://play-lh.googleusercontent.com/8s7VpaT4BJgaTq8u7eF2FIgW3-gOZzdOYRvI6-edXj2mpJms_XZS8ULxbe1VSd8RoVs\n   - **Description:** A beautifully designed adventure game where you help Josef the robot save his girlfriend from the Black Cap Brotherhood gang.\n\n4. **Cat Legends: Idle RPG Games**\n   - **Image URL:** https://play-lh.go

In [22]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate



In [23]:
template = """Answer the question in your own words from the 
    context given to you.
    If questions are asked where there is no relevant context available, please answer from 
    what you know.

    Context: {context}

    Human: {question}
    Assistant:"""

prompt = PromptTemplate(
input_variables=["context",  "question"], template=template)


In [24]:

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

memory.save_context({"input": "Which games may you recommend for adventure lover give me 10 recommendation?, give me image url too"},
                {"output": """ Here are 10 adventure games with image URLs:

1. **Klondike Adventures**
   - Image URL: https://play-lh.googleusercontent.com/jW61KFsluyqSfGV_GhBNWdNJfmNfF8vApum1WiWw2hNR7cirY4hIaGd6fQ2zUGomRXw
   - Combines adventure, exploration, and city-building.

2. **Mythic Samkok**
   - Image URL: https://play-lh.googleusercontent.com/C4gz0QN80HKvBzoEndxCPj8FaKWYfauoztkZH4VAnDVaHjBsZUFMQGfdjQb7fejNs0d8
   - Embark on an epic journey through the Three Kingdoms.

3. **Fortias Saga: Action Adventure**
   - Image URL: https://play-lh.googleusercontent.com/-T2g7Lcbim190MJ_g7qnZmTn7zjpdrkryPeHSanFq81Je5MJUeG_0ah3KiqdesC_LeU
   - Battle monsters and recruit heroes in a nostalgic art style.

4. **Clash of Clans**
   - Image URL: https://play-lh.googleusercontent.com/LByrur1mTmPeNr0ljI-uAUcct1rzmTve5Esau1SwoAzjBXQUby6uHIfHbF9TAT51mgHm
   - Build your village, raise a clan, and compete in epic Clan Wars.

5. **Pokémon GO** (Not in provided context, but a popular choice)
   - Image URL: Search for "Pokémon GO logo" on Google Images.
   - Explore the real world to catch Pokémon.

6. **The Legend of Zelda: Breath of the Wild** (Not on Play Store, but a console/PC classic)
   - Image URL: Search for "Breath of the Wild logo" on Google Images.
   - Explore a vast open world filled with adventure and puzzles.

7. **Minecraft** (Available on Play Store)
   - Image URL: Search for "Minecraft logo" on Google Images.
   - Build, explore, and survive in a procedurally generated world.

8. **Stardew Valley** (Available on Play Store)
   - Image URL: Search for "Stardew Valley logo" on Google Images.
   - Build a farm, explore a charming town, and uncover its secrets.

9. **Terraria** (Available on Play Store)
   - Image URL: Search for "Terraria logo" on Google Images.
   - Dig, fight, explore, and build in a 2D action-adventure sandbox.

10. **Genshin Impact** (Available on Play Store)
    - Image URL: Search for "Genshin Impact logo" on Google Images.
    - Explore a vast open world with stunning visuals and engaging combat.


                 """
                 })


In [25]:

qa = RetrievalQA.from_chain_type(llm=model, retriever=retriever1, memory=memory,chain_type_kwargs={'prompt': prompt}
)

In [26]:
# qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=retriever1)
qa.invoke({"query": "Which games may you recommend for adventure lover?"} )


{'query': 'Which games may you recommend for adventure lover?',
 'chat_history': [HumanMessage(content='Which games may you recommend for adventure lover give me 10 recommendation?, give me image url too'),
  AIMessage(content=' Here are 10 adventure games with image URLs:\n\n1. **Klondike Adventures**\n   - Image URL: https://play-lh.googleusercontent.com/jW61KFsluyqSfGV_GhBNWdNJfmNfF8vApum1WiWw2hNR7cirY4hIaGd6fQ2zUGomRXw\n   - Combines adventure, exploration, and city-building.\n\n2. **Mythic Samkok**\n   - Image URL: https://play-lh.googleusercontent.com/C4gz0QN80HKvBzoEndxCPj8FaKWYfauoztkZH4VAnDVaHjBsZUFMQGfdjQb7fejNs0d8\n   - Embark on an epic journey through the Three Kingdoms.\n\n3. **Fortias Saga: Action Adventure**\n   - Image URL: https://play-lh.googleusercontent.com/-T2g7Lcbim190MJ_g7qnZmTn7zjpdrkryPeHSanFq81Je5MJUeG_0ah3KiqdesC_LeU\n   - Battle monsters and recruit heroes in a nostalgic art style.\n\n4. **Clash of Clans**\n   - Image URL: https://play-lh.googleusercontent.

In [28]:
res = qa.invoke({"query": "Which games may you recommend for adventure lover give me 10 recommendation?, give me image url too"} )


In [29]:
res["result"]

'Here are 10 adventure games for you, combining the provided context with general knowledge:\n\n1. **Dungeons of Dreadrock:** A puzzle-filled dungeon crawler with a unique art style. \n   * Image URL: https://play-lh.googleusercontent.com/0JGFClzQlrsu159aB2R0xaiITlZbAZeQuXe8XOP4xJkuilfzN47S4t22ezVvLaD2KA\n\n2. **Shovel Pirate:** A charming platformer with treasure-hunting and light puzzle elements.\n   * Image URL: https://play-lh.googleusercontent.com/bz_SqMcnNxw0qO_qXR2KwOoTi3mw3610_xUgeJJJtFyKfsJItEx9Ue0OJXMYk_wIpQ\n\n3. **Machinarium:** A beautifully crafted point-and-click adventure set in a robot world.\n   * Image URL: https://play-lh.googleusercontent.com/x9uKZ2K7l_bwHliNpPO6fzBxzQBWXN4JAZSxkZ5lpINZKCHBVOcalI3SKSiOozwGYXA\n\n4. **Cat Legends: Idle RPG Games:** Embark on a quest with adorable cats in this idle RPG.\n   * Image URL: https://play-lh.googleusercontent.com/J6oGx6rOZvF9pq1rbAm5BUcvHmR5jNmMCpi7by-vCvfoPhRKc50E5BnfmgtQd7LyHgc\n\n5. **Stardew Valley:** (General knowledg